## 1. Clasificación de barrio en el que Andres vive

In [1]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
lat =4.6841502
long = -74.0440973

In [3]:
map_bogota = folium.Map(location = [lat, long], zoom_start = 60)

folium.CircleMarker(
        [lat, long],
        radius=5,
        popup= 'El pedregal',
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bogota)  
    
map_bogota
                       

#### Foursquare credenciales y versión

In [4]:
CLIENT_ID = 'OJBX11LZBW5JWYI3YQMH0D4KBR1FP4W3OG5QQB05335OWJEC' # your Foursquare ID
CLIENT_SECRET = 'AOGSKEWATG0JF1PDZSXIO4YQBKLNBDUCYTSDSNL5A3L1PXWN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OJBX11LZBW5JWYI3YQMH0D4KBR1FP4W3OG5QQB05335OWJEC
CLIENT_SECRET:AOGSKEWATG0JF1PDZSXIO4YQBKLNBDUCYTSDSNL5A3L1PXWN


## Obtenemos el top 100 de lugares que existen en este barrio dentro de un radio de 500 mts

In [5]:
#definimos la url request
limit = 100
radio = 500

url ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, radio, limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=OJBX11LZBW5JWYI3YQMH0D4KBR1FP4W3OG5QQB05335OWJEC&client_secret=AOGSKEWATG0JF1PDZSXIO4YQBKLNBDUCYTSDSNL5A3L1PXWN&v=20180605&ll=4.6841502,-74.0440973&radius=500&limit=100'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f470fdfc7db236072556853'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Usaquén',
  'headerFullLocation': 'Usaquén, Bogotá',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 72,
  'suggestedBounds': {'ne': {'lat': 4.6886502045000045,
    'lng': -74.03959064536639},
   'sw': {'lat': 4.679650195499996, 'lng': -74.04860395463362}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5303fc7311d2955289759b50',
       'name': 'Good Club de Bienestar',
       'location': {'address': 'Cra 11 A # 98 -53',
        'lat': 4.682004106716711,
        'lng': -74.04479362180517,
        'labeledLatLngs': [{'label': 'display',
          'lat': 

In [7]:
# funcion que extrae la categoria del lugar
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

c:\users\carmen\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Good Club de Bienestar,Gym / Fitness Center,4.682004,-74.044794
1,"Tierra del Fuego, Choripan Argentino",Argentinian Restaurant,4.682582,-74.044016
2,Arepa Cool,Latin American Restaurant,4.683576,-74.047218
3,Hamburguesas El Corral,Burger Joint,4.681437,-74.044795
4,Café 18,Breakfast Spot,4.680554,-74.046028
...,...,...,...,...
67,BIGMAMMA BURGER,Burger Joint,4.682719,-74.048128
68,Sopitas & Frijolada 98,Soup Place,4.683481,-74.047472
69,Exito Express,Shop & Service,4.681161,-74.041927
70,La Brasa Roja,Fast Food Restaurant,4.680811,-74.041489


In [9]:
print('{} venues were returned'.format(nearby_venues.shape[0]))

72 venues were returned


In [10]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 42 uniques categories.


In [11]:
pedregal_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")
pedregal_onehot['Neighborhood'] = 'pedregal'
# move neighborhood column to the first column
fixed_columns = [pedregal_onehot.columns[-1]] + list(pedregal_onehot.columns[:-1])
pedregal_onehot = pedregal_onehot[fixed_columns]
pedregal_onehot.head()

,Neighborhood,Animal Shelter,Argentinian Restaurant,Asian Restaurant,Bakery,Bistro,Breakfast Spot,Burger Joint,Burrito Place,Café,...,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shop & Service,Soup Place,Sushi Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio
0,pedregal,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,pedregal,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,pedregal,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,pedregal,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,pedregal,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
pedregal_onehot.shape

(72, 43)

In [13]:
pedregal_grouped = pedregal_onehot.groupby('Neighborhood').mean().reset_index()
pedregal_grouped

,Neighborhood,Animal Shelter,Argentinian Restaurant,Asian Restaurant,Bakery,Bistro,Breakfast Spot,Burger Joint,Burrito Place,Café,...,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shop & Service,Soup Place,Sushi Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio
0,pedregal,0.013889,0.013889,0.013889,0.013889,0.013889,0.027778,0.111111,0.013889,0.083333,...,0.041667,0.013889,0.041667,0.013889,0.027778,0.013889,0.013889,0.013889,0.013889,0.013889


In [14]:
pedregal_grouped.shape

(1, 43)

In [15]:
num_top_venues = 20

for hood in pedregal_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = pedregal_grouped[pedregal_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----pedregal----
                        venue  freq
0                Burger Joint  0.11
1                        Café  0.08
2                       Hotel  0.07
3         Japanese Restaurant  0.04
4                 Coffee Shop  0.04
5                  Restaurant  0.04
6                 Pizza Place  0.04
7        Gym / Fitness Center  0.03
8                   Juice Bar  0.03
9              Breakfast Spot  0.03
10  Latin American Restaurant  0.03
11         Seafood Restaurant  0.03
12    Comfort Food Restaurant  0.03
13        Peruvian Restaurant  0.01
14         Mexican Restaurant  0.01
15            Motorcycle Shop  0.01
16             Animal Shelter  0.01
17                        Pub  0.01
18              Hot Dog Joint  0.01
19             Shop & Service  0.01




## 2. Analizamos los barrios de Brooklyn - Nueva York.

In [16]:
with open('nyc.json') as json_data:
    newyork_data = json.load(json_data)

In [17]:
neighborhoods_data = newyork_data['features']

In [18]:
#revisamos el primer barrio de nueva york
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Llevarlo a pandas

In [19]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

loop through the data and fill the dataframe one row at a time.

In [20]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [21]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Obtener latitude y longitud de Brooklyn, NY.

In [22]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [23]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471
5,Brooklyn,Brighton Beach,40.576825,-73.965094
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438
8,Brooklyn,Flatbush,40.636326,-73.958401
9,Brooklyn,Crown Heights,40.670829,-73.943291


Definir las credenciales y version de foursquare

In [24]:
CLIENT_ID = 'OJBX11LZBW5JWYI3YQMH0D4KBR1FP4W3OG5QQB05335OWJEC' # your Foursquare ID
CLIENT_SECRET = 'AOGSKEWATG0JF1PDZSXIO4YQBKLNBDUCYTSDSNL5A3L1PXWN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [25]:
brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

In [26]:
neighborhood_latitude = brooklyn_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = brooklyn_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = brooklyn_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [27]:
#obtengamos el top 100 de lugares en wakefield

In [28]:
limit = 100
radio = 500

url ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radio, limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=OJBX11LZBW5JWYI3YQMH0D4KBR1FP4W3OG5QQB05335OWJEC&client_secret=AOGSKEWATG0JF1PDZSXIO4YQBKLNBDUCYTSDSNL5A3L1PXWN&v=20180605&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f470e51412808288dd73333'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 80,
  'suggestedBounds': {'ne': {'lat': 40.63030106951066,
    'lng': -74.02470273356597},
   'sw': {'lat': 40.62130106051065, 'lng': -74.03653865351028}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b895827f964a5206c2d32e3',
       'name': 'Pilo Arts Day Spa and Salon',
       'location': {'address': '8412 3rd Ave',
        'lat': 40.62474788273414,
        'lng': -74.03059056940135,
        'labeledLatL

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

c:\users\carmen\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Cocoa Grinder,Juice Bar,40.623967,-74.030863
3,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
4,Leo's Casa Calamari,Pizza Place,40.624200,-74.030931


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

80 venues were returned by Foursquare.


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )



Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [35]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2775, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place


In [36]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,49,49,49,49,49,49
Bay Ridge,80,80,80,80,80,80
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,31,31,31,31,31,31
Bergen Beach,5,5,5,5,5,5
...,...,...,...,...,...,...
Vinegar Hill,28,28,28,28,28,28
Weeksville,15,15,15,15,15,15
Williamsburg,34,34,34,34,34,34


In [37]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 292 uniques categories.


## Analisis de cada barrio en Brooklyn

In [38]:
# one hot encoding
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
brooklyn_onehot.head()

,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Sto

In [39]:
#eliminamos la Columna con nombre 'Neighborhood' y corroboramos
brooklyn_onehot.drop(['Neighborhood'], axis=1, inplace = True)
brooklyn_onehot.head()

,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Re

In [40]:
# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Sto

In [41]:
brooklyn_onehot.shape

(2775, 292)

In [42]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Sto

In [43]:
num_top_venues = 5
a=[]
for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0           Pizza Place  0.06
1    Chinese Restaurant  0.06
2           Gas Station  0.04
3          Dessert Shop  0.04
4  Fast Food Restaurant  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.06
1                  Spa  0.05
2     Greek Restaurant  0.05
3          Pizza Place  0.05
4  American Restaurant  0.04


----Bedford Stuyvesant----
           venue  freq
0    Coffee Shop  0.11
1    Pizza Place  0.07
2           Café  0.07
3  Deli / Bodega  0.07
4            Bar  0.07


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.10
1          Donut Shop  0.06
2       Grocery Store  0.06
3  Italian Restaurant  0.06
4    Sushi Restaurant  0.06


----Bergen Beach----
                  venue  freq
0       Harbor / Marina   0.4
1    Athletics & Sports   0.2
2        Baseball Field   0.2
3            Playground   0.2
4  Pakistani Restaurant   0.0


----Boerum Hill----
                    venue

4     Candy Store  0.09


----Mill Basin----
                 venue  freq
0          Pizza Place  0.09
1  Japanese Restaurant  0.09
2   Chinese Restaurant  0.06
3                 Bank  0.06
4       Cosmetics Shop  0.06


----Mill Island----
                   venue  freq
0                   Pool   1.0
1      Accessories Store   0.0
2            Opera House   0.0
3           Outlet Store   0.0
4  Outdoors & Recreation   0.0


----New Lots----
                 venue  freq
0             Pharmacy  0.11
1  Fried Chicken Joint  0.11
2   Chinese Restaurant  0.11
3          Pizza Place  0.11
4                Plaza  0.05


----North Side----
                 venue  freq
0          Coffee Shop  0.10
1          Pizza Place  0.06
2             Wine Bar  0.05
3          Yoga Studio  0.03
4  American Restaurant  0.03


----Ocean Hill----
                 venue  freq
0        Grocery Store  0.13
1  Fried Chicken Joint  0.10
2        Deli / Bodega  0.10
3             Bus Stop  0.06
4                 F

Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [45]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Bath Beach,Pizza Place,Chinese Restaurant,Dessert Shop,Pharmacy,Sushi Restaurant,Donut Shop,Fast Food Restaurant,Gas Station,Italian Restaurant,Bubble Tea Shop,Cantonese Restaurant,Sandwich Place,Russian Restaurant,Burger Joint,Restaurant,Coffee Shop,Spanish Restaurant,Accessories Store,Diner,Dim Sum Restaurant
1,Bay Ridge,Italian Restaurant,Pizza Place,Spa,Greek Restaurant,Pharmacy,Bar,American Restaurant,Chinese Restaurant,Café,Sandwich Place,Sushi Restaurant,Hookah Bar,Playground,Bagel Shop,Ice Cream Shop,Grocery Store,Breakfast Spot,Kids Store,Bookstore,Boutique
2,Bedford Stuyvesant,Coffee Shop,Pizza Place,Café,Deli / Bodega,Bar,BBQ Joint,Gourmet Shop,Cocktail Bar,Basketball Court,Boutique,Thrift / Vintage Store,Juice Bar,Tiki Bar,Gift Shop,Bagel Shop,Park,Wine Shop,Japanese Restaurant,Italian Restaurant,Wine Bar
3,Bensonhurst,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Grocery Store,Shabu-Shabu Restaurant,Supermarket,Spa,Food,Butcher,Flower Shop,Hotpot Restaurant,Cha Chaan Teng,Noodle House,Coffee Shop,Cosmetics Shop,Park,Pet Store,Liquor Store
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Athletics & Sports,Fish Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Flea Market,Falafel Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Farm


# Cluster Neighborhoods in Brooklyn

Run k-means to cluster the neighborhood into 5 clusters.

usamos el metodo del codo para seleccionar el numero de clusters que vamos a usar

In [46]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

### GENERAMOS LA CLASIFICACION DE LOS BARRIOS CON K-MEANS

In [47]:
# set number of clusters
kclusters = 6

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([1, 5, 5, 5, 5, 5, 1, 1, 1, 5, 1, 5, 0, 5, 1, 5, 5, 1, 5, 1, 1, 5,
       5, 5, 1, 1, 5, 1, 1, 1, 5, 5, 5, 1, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5,
       5, 5, 5, 3, 1, 5, 1, 5, 4, 5, 5, 5, 1, 5, 1, 1, 2, 5, 5, 1, 1, 5,
       1, 5, 5, 1])

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,5,Italian Restaurant,Pizza Place,Spa,Greek Restaurant,Pharmacy,Bar,American Restaurant,Chinese Restaurant,Café,Sandwich Place,Sushi Restaurant,Hookah Bar,Playground,Bagel Shop,Ice Cream Shop,Grocery Store,Breakfast Spot,Kids Store,Bookstore,Boutique
1,Brooklyn,Bensonhurst,40.611009,-73.995180,5,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Grocery Store,Shabu-Shabu Restaurant,Supermarket,Spa,Food,Butcher,Flower Shop,Hotpot Restaurant,Cha Chaan Teng,Noodle House,Coffee Shop,Cosmetics Shop,Park,Pet Store,Liquor Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Mexican Restaurant,Bakery,Mobile Phone Shop,Pizza Place,Bank,Latin American Restaurant,Ice Cream Shop,Pharmacy,Gym,Fried Chicken Joint,Deli / Bodega,Breakfast Spot,Grocery Store,Sandwich Place,Women's Store,Record Shop,Supplement Shop,Creperie,Café,Italian Restaurant
3,Brooklyn,Greenpoint,40.730201,-73.954241,5,Bar,Pizza Place,Cocktail Bar,Coffee Shop,Deli / Bodega,Mexican Restaurant,French Restaurant,Yoga Studio,Flower Shop,Café,Bakery,Nail Salon,Thrift / Vintage Store,Salon / Barbershop,Tea Room,Sushi Restaurant,Restaurant,Italian Restaurant,Polish Restaurant,Record Shop
4,Brooklyn,Gravesend,40.595260,-73.973471,5,Lounge,Pizza Place,Chinese Restaurant,Italian Restaurant,Bakery,Restaurant,Farmers Market,Martial Arts School,Gym,Bar,Men's Store,Electronics Store,Diner,Furniture / Home Store,Pharmacy,Donut Shop,Record Shop,Music Venue,Metro Station,Flea Market


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 3. Predecimos el cluster al que pertenece el barrio de Andres

In [51]:
#pedregal_grouped_clustering = pedregal_grouped.drop('Neighborhood', 1)

#kmeans_pedregal = kmeans.predict(pedregal_grouped_clustering)
#kmeans.labels_[0:100] 

In [52]:
pedregal_columns = pedregal_grouped_clustering.columns.values.tolist()

In [53]:
brooklyn_columns = brooklyn_grouped_clustering.columns.values.tolist()

In [54]:
i = 1
j = 1
for column in brooklyn_columns:
    if column in pedregal_columns:
        print ('Yes/', i)
        i+=1
    else:
        print ('No/', j)
        j+=1
    

No/ 1
No/ 2
No/ 3
No/ 4
No/ 5
No/ 6
Yes/ 1
No/ 7
No/ 8
No/ 9
Yes/ 2
No/ 10
No/ 11
No/ 12
Yes/ 3
No/ 13
No/ 14
No/ 15
No/ 16
No/ 17
No/ 18
No/ 19
No/ 20
No/ 21
No/ 22
No/ 23
No/ 24
No/ 25
Yes/ 4
No/ 26
No/ 27
No/ 28
No/ 29
No/ 30
Yes/ 5
No/ 31
No/ 32
No/ 33
No/ 34
Yes/ 6
Yes/ 7
No/ 35
No/ 36
No/ 37
No/ 38
No/ 39
Yes/ 8
No/ 40
No/ 41
No/ 42
Yes/ 9
No/ 43
No/ 44
No/ 45
No/ 46
No/ 47
No/ 48
No/ 49
No/ 50
No/ 51
No/ 52
Yes/ 10
No/ 53
No/ 54
Yes/ 11
No/ 55
Yes/ 12
No/ 56
No/ 57
No/ 58
No/ 59
Yes/ 13
No/ 60
No/ 61
No/ 62
No/ 63
Yes/ 14
No/ 64
No/ 65
No/ 66
No/ 67
Yes/ 15
No/ 68
No/ 69
No/ 70
No/ 71
No/ 72
No/ 73
No/ 74
No/ 75
Yes/ 16
No/ 76
No/ 77
Yes/ 17
No/ 78
No/ 79
Yes/ 18
No/ 80
No/ 81
No/ 82
No/ 83
No/ 84
No/ 85
No/ 86
No/ 87
Yes/ 19
No/ 88
No/ 89
No/ 90
No/ 91
No/ 92
No/ 93
No/ 94
No/ 95
No/ 96
No/ 97
No/ 98
Yes/ 20
No/ 99
No/ 100
No/ 101
No/ 102
No/ 103
No/ 104
Yes/ 21
No/ 105
No/ 106
No/ 107
No/ 108
No/ 109
No/ 110
No/ 111
No/ 112
No/ 113
No/ 114
No/ 115
No/ 116
No/ 117
Yes/ 22
No/ 1

## De las 290 venues en Brooklyn 254 no existen en el barrio el pedregal, y 36 si existen.

In [66]:
#print(pedregal_columns)

In [67]:
#print(brooklyn_columns)

In [57]:
k = 1
l = 1
intento1 = pedregal_grouped_clustering
for column in pedregal_columns:
    if column not in brooklyn_columns:
        #print (column,'Yes/', k)
        intento1.drop(column, axis = 1, inplace = True )
        k+=1
    #else:
     #   print (column, 'No/', l)
      #  l+=1

In [58]:
intento1

,Argentinian Restaurant,Asian Restaurant,Bakery,Bistro,Breakfast Spot,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Coffee Shop,Convenience Store,Coworking Space,Deli / Bodega,Diner,Donut Shop,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,German Restaurant,Gym / Fitness Center,Hotel,Japanese Restaurant,Juice Bar,Latin American Restaurant,Mexican Restaurant,Motorcycle Shop,Peruvian Restaurant,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Soup Place,Sushi Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio
0,0.013889,0.013889,0.013889,0.013889,0.027778,0.111111,0.013889,0.083333,0.013889,0.041667,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.027778,0.069444,0.041667,0.027778,0.027778,0.013889,0.013889,0.013889,0.041667,0.013889,0.041667,0.013889,0.027778,0.013889,0.013889,0.013889,0.013889


## de las 42 variables (venues) que existen en el barrio de Andres (El pedregal), 36 existen en los barrios de Brooklyn.

dado que para proyectar el cluster tenemos que tener la misma cantidad de variables debemos agrear las que 

In [59]:
agregar = (i + j) - (k + l)
print(agregar)

286


In [60]:
i = 1
j = 1
for column in brooklyn_columns:
    if column not in pedregal_columns:
        #print ('Yes/', i)
        intento1[column] = 0
        i+=1
    #else:
     #   print ('No/', j)
      #  j+=1
    

In [61]:
intento1

,Argentinian Restaurant,Asian Restaurant,Bakery,Bistro,Breakfast Spot,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Coffee Shop,Convenience Store,Coworking Space,Deli / Bodega,Diner,Donut Shop,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,German Restaurant,Gym / Fitness Center,Hotel,Japanese Restaurant,Juice Bar,Latin American Restaurant,Mexican Restaurant,Motorcycle Shop,Peruvian Restaurant,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Soup Place,Sushi Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,BBQ Joint,Bagel Shop,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brewery,Bridge,Bubble Tea Shop,Buffet,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Concert Hall,Construction & Landscaping,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Distillery,Dive Bar,Dog Run,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Factory,Farm,Farmers Market,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Scenic Lookout,School,Sculpture Garden,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / At

In [62]:
kmeans_pedregal = kmeans.predict(intento1)
kmeans_pedregal[0]

5

In [63]:
barrios_df = brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == kmeans_pedregal[0], brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

In [64]:
barrios_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Bay Ridge,Italian Restaurant,Pizza Place,Spa,Greek Restaurant,Pharmacy,Bar,American Restaurant,Chinese Restaurant,Café,Sandwich Place,Sushi Restaurant,Hookah Bar,Playground,Bagel Shop,Ice Cream Shop,Grocery Store,Breakfast Spot,Kids Store,Bookstore,Boutique
1,Bensonhurst,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Grocery Store,Shabu-Shabu Restaurant,Supermarket,Spa,Food,Butcher,Flower Shop,Hotpot Restaurant,Cha Chaan Teng,Noodle House,Coffee Shop,Cosmetics Shop,Park,Pet Store,Liquor Store
3,Greenpoint,Bar,Pizza Place,Cocktail Bar,Coffee Shop,Deli / Bodega,Mexican Restaurant,French Restaurant,Yoga Studio,Flower Shop,Café,Bakery,Nail Salon,Thrift / Vintage Store,Salon / Barbershop,Tea Room,Sushi Restaurant,Restaurant,Italian Restaurant,Polish Restaurant,Record Shop
4,Gravesend,Lounge,Pizza Place,Chinese Restaurant,Italian Restaurant,Bakery,Restaurant,Farmers Market,Martial Arts School,Gym,Bar,Men's Store,Electronics Store,Diner,Furniture / Home Store,Pharmacy,Donut Shop,Record Shop,Music Venue,Metro Station,Flea Market
6,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Sandwich Place,Yoga Studio,Pizza Place,Russian Restaurant,Restaurant,Creperie,Diner,Buffet,Park,Outlet Store,Miscellaneous Shop,Grocery Store,Karaoke Bar,Italian Restaurant,Café,Hotel,American Restaurant,Bank
7,Manhattan Terrace,Pizza Place,Ice Cream Shop,Donut Shop,Bank,Eastern European Restaurant,Cosmetics Shop,Convenience Store,Coffee Shop,Organic Grocery,Chinese Restaurant,Bus Station,Mobile Phone Shop,Burger Joint,Steakhouse,Liquor Store,Grocery Store,Spa,Jazz Club,Bagel Shop,Japanese Restaurant
9,Crown Heights,Pizza Place,Café,Museum,Bookstore,Kosher Restaurant,Deli / Bodega,Pharmacy,Coffee Shop,Salon / Barbershop,Candy Store,Moving Target,Burger Joint,Playground,Supermarket,Liquor Store,Fried Chicken Joint,Bakery,Bagel Shop,Halal Restaurant,Field
11,Kensington,Grocery Store,Thai Restaurant,Pizza Place,Ice Cream Shop,Mexican Restaurant,Café,Furniture / Home Store,Liquor Store,Lingerie Store,Sandwich Place,Gas Station,Nail Salon,Coffee Shop,Spa,Outdoors & Recreation,Music Venue,Donut Shop,Bakery,Bagel Shop,Japanese Restaurant
12,Windsor Terrace,Deli / Bodega,Diner,Park,Plaza,Grocery Store,Café,Food Truck,Middle Eastern Restaurant,Memorial Site,Chinese Restaurant,Bookstore,Sushi Restaurant,Beer Store,Butcher,Bar,Thrift / Vintage Store,French Restaurant,Antique Shop,Bagel Shop,Wine Shop
13,Prospect Heights,Bar,Mexican Restaurant,Wine Shop,Thai Restaurant,Bakery,Cocktail Bar,Restaurant,Beer Bar,Pizza Place,New American Restaurant,Café,Gourmet Shop,Ice Cream Shop,Southern / Soul Food Restaurant,Diner,Wine Bar,Coffee Shop,American Restaurant,Rental Car Location,Farmers Market


In [65]:
barrios_df[['Neighborhood']].reset_index(drop = True)

,Neighborhood
0,Bay Ridge
1,Bensonhurst
2,Greenpoint
3,Gravesend
4,Sheepshead Bay
5,Manhattan Terrace
6,Crown Heights
7,Kensington
8,Windsor Terrace
9,Prospect Heights
